In [9]:
%load_ext autoreload
%autoreload 2
import html_parser
from ner import EntityExtractor
import psycopg2
import pandas as pd
import glob
from tqdm import tqdm
from names_parser import NameParser
import db
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

tqdm.pandas()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
docs, cols = db.fetch_228(limit=1000000)
print("Total docs: ", len(docs))

Total docs:  340198


In [6]:
df = pd.DataFrame(docs, columns=cols)
df["case_year_result"] = df["case_year_result"].apply(lambda x: str(int(x)))
df.drop(
    labels=df[~df["case_common_document_type"].isin(
        ['Приговор', 'Приговоры', 'Постановление', 'Постановления']
        )].index,
    inplace=True
    )
print(len(df))

339023


In [10]:
uid = '7ce60d022f887529bb4280215147907d'
text = db.load_text(uid)
text[:100]

'№ 1-147/2017\nПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nг. Екатеринбург \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0'

In [24]:
segmenter = Segmenter()
doc = Doc(text.replace("\n", " "))
doc.segment(segmenter)
# print(doc.tokens[:5])
print(doc.sents[:5])

[DocSent(stop=1089, text='№ 1-147/2017 ПРИГОВОР ИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ..., tokens=[...]), DocSent(start=1091, stop=1166, text='Преступление им совершено в г. Екатеринбурге при ..., tokens=[...]), DocSent(start=1167, stop=1399, text='18.11.2017 года не позднее 08 часов 13 минут у Ве..., tokens=[...]), DocSent(start=1401, stop=1935, text='18.11.2017 года в период времени с 14 часов 30 ми..., tokens=[...]), DocSent(start=1936, stop=2143, text='После оплаты в этот же день Вершинин А.Е. от лица..., tokens=[...])]


In [28]:
sents = [d.text for d in doc.sents]
for s in sents:
    print(s)
    print("\n--------------------------------------------------------")

№ 1-147/2017 ПРИГОВОР ИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ г. Екатеринбург                                                                                               19 марта 2018 года Орджоникидзевский районный суд г. Екатеринбурга в составе:председательствующего судьи Галимовой Н.Р.,  с участием государственного обвинителя - помощника прокурора Орджоникидзевского района г. Екатеринбурга Власовой К.В., подсудимого Вершинина А.Е., защитника в лице адвоката Солдаткина В.А., при секретаре Бусыгиной А.А., рассмотрев в открытом судебном заседании материалы уголовного дела в отношении: Вершинина А. Е., < данные изъяты >, ранее судимого: -13.12.2016 года приговором Нижнесергинского районного суда Свердловской области по ч. 1 ст. 228 Уголовного кодекса Российской Федерации к наказанию в виде штрафа в размере 10 000 рублей (наказание отбыто < дд.мм.гггг >), обвиняемого в совершении преступления, предусмотренного ч. 2 ст. 228 Уголовного кодекса Российской Федерации, УСТАНОВИЛ: Вершинин А.Е. совершил 